In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
import copy

import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn_rvm import EMRVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from sklearn.svm import OneClassSVM
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest 
from sklearn.neighbors import LocalOutlierFactor

# 🌠 Anomaly Detection for Supervised Task

- Anoamly Detection은 근본적인 Supervised Classification Task를 Anoamly Detection으로 변경한 Task에여대하여 Anomaly Detection이 가능할까?

# Load Dataset

In [133]:
# import statsmodels.api as sm
# from tabulate import tabulate

# ces11 = sm.datasets.get_rdataset('CES11', 'carData')

# ces11.data.sample(12)

In [134]:
# random_seed
# rand_seed = 1024 
rand_seed = 2048 
# rand_seed = 4096

# Select Dataset

# dataset_name = 'diabetes'
# dataset_name = 'breast_cancer'
dataset_name = 'digits'

if dataset_name == 'diabetes':
    df = pd.read_csv('diabetes.csv')
    X = df.iloc[:,:-1].values   
    y = df.iloc[:,-1].values    
    THRESHOLD_FOR_AUTOENCODER = 0.07
    THRESHOLD_FOR_DENSITY=36
    NUM_1ST_HIDDEN = 6 
    NUM_2ND_HIDDEN = 3 
    class_anomaly = 1

elif dataset_name == 'breast_cancer':
    breast_cancer = datasets.load_breast_cancer()
    X = breast_cancer.data
    y = breast_cancer.target
    THRESHOLD_FOR_AUTOENCODER = 0.89 
    THRESHOLD_FOR_DENSITY = 70
    NUM_1ST_HIDDEN = 32 
    NUM_2ND_HIDDEN = 8 
    class_anomaly = 1

elif dataset_name == 'digits':
    digits = datasets.load_digits()
    X = digits.data
    y = digits.target

    THRESHOLD_FOR_AUTOENCODER = 8.5
    THRESHOLD_FOR_DENSITY = 23
    NUM_1ST_HIDDEN = 32 
    NUM_2ND_HIDDEN = 8 

    class_anomaly = 5

else:
    pass

In [135]:
print(f'Median {np.median(y)}')
print(f'Min {np.min(y)}')
print(f'Max {np.max(y)}')

Median 4.0
Min 0
Max 9


# Split data by specific threshold
- To convert regression problem to classification problem (two class for anomaly detection)

In [136]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)

X shape
(1797, 64)
Y shape
(1797,)


In [137]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=rand_seed, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=rand_seed, train_size=0.8)



In [138]:
y_train_a = y_train.copy()
y_valid_a = y_valid.copy()
y_test_a = y_test.copy()


y_train_a[y_train==class_anomaly] = -1
y_train_a[y_train!=class_anomaly] = 1

y_valid_a[y_valid==class_anomaly] = -1
y_valid_a[y_valid!=class_anomaly] = 1

y_test_a[y_test==class_anomaly] = -1
y_test_a[y_test!=class_anomaly] = 1

In [139]:
# scaling by minmax scaler
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [140]:
print('valid', len(y_valid_a[y_valid_a==-1]))
print('train', len(y_train_a[y_train_a ==-1]))
print('test', len(y_test_a[y_test_a ==-1]))

print('len train origin : ', len(x_train))
x_train_only = x_train[y_train_a==1]
print('len train origin with only true : ', len(x_train_only))

valid 22
train 111
test 49
len train origin :  1149
len train origin with only true :  1038


# SVM (Support Vector Classification)
- Classification 

In [141]:
param_grid = [
    {'kernel': ['linear'], 'C': [1.0, 2.0, 3.0, 10.]},
    {'kernel': ['rbf'], 'C': [1.0, 2.0, 3.0, 5.0, 10.],
    'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

elapsed_time_kernel_svm = []

svm_classifier = SVC(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

start_time = datetime.now()
grid_search = GridSearchCV(svm_classifier, param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_svc_classifier = grid_search.fit(x_train, y_train_a)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())



# y_pred = svm_classifier.predict(x_test)
# start_time = datetime.now()
# y_pred = best_svm_classifier.predict(x_test)
# elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
# acc_svm_kernel = accuracy_score(y_test, y_pred)

Fitting 7 folds for each of 34 candidates, totalling 238 fits
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, 

In [142]:

# y_pred = svm_classifier.predict(x_test)
start_time = datetime.now()
y_pred = best_svc_classifier.predict(x_test)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
acc_svm_kernel = accuracy_score(y_test_a, y_pred)



In [143]:

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred))
print('Best Prameters ', grid_search.best_params_)
print('Accuracy ', acc_svm_kernel)
print('Elapsed Time(train, test) ', elapsed_time_kernel_svm)

Confusion Matrix
 [[ 49   0]
 [  0 311]]
Best Prameters  {'C': 3.0, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy  1.0
Elapsed Time(train, test)  [4.318349, 0.002833]


# One-Class SVM - Linear & Kernel

In [144]:
param_grid = [
    {'kernel': ['linear'], 'nu': [0.1, 0.25, 0,5, 0.7]},
    {'kernel': ['rbf'], 'nu': [0.1, 0.25, 0,5, 0.7],
    'gamma': [0.01, 0.03, 0.1, 0.3, 1.0]},
]

elapsed_time_kernel_svm = []

svm_classifier = OneClassSVM(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

start_time = datetime.now()
grid_search = GridSearchCV(svm_classifier, param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train_only)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())


start_time = datetime.now()
y_pred = best_svm_classifier.predict(x_test)
elapsed_time_kernel_svm.append((datetime.now()-start_time).total_seconds())
acc_svm_kernel = accuracy_score(y_test_a, y_pred)



Fitting 7 folds for each of 30 candidates, totalling 210 fits
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END ..............................kernel=linear, nu=0.1; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=linear, nu=0.25; total time=   0.0s
[CV] END .............................kernel=li

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ..............................kernel=linear, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ....................gamma=0.01, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END ....................gamma=0.01, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END ....................gamma=0.01, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END ....................gamma=0.01, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.01, kernel=rbf, nu=0.7; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=0.03, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ...................

...

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ......................gamma=1.0, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=1.0, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END ......................gamma=1.0, kernel=rbf, nu=0.1; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END .....................gamma=1.0, kernel=rbf, nu=0.25; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=0; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=5; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=5; total time=   0.0s
[CV] END ........................gamma=1.0, kernel=rbf, nu=5; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

In [145]:
print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred))
print('Best Prameters ', grid_search.best_params_)
print('Accuracy ', acc_svm_kernel)
print('Elapsed Time(train, test) ', elapsed_time_kernel_svm)
# Isolation Forest 

Confusion Matrix
 [[  3  46]
 [ 27 284]]
Best Prameters  {'kernel': 'linear', 'nu': 0.1}
Accuracy  0.7972222222222223
Elapsed Time(train, test)  [2.038782, 0.0]


# Isolation Forest

In [146]:
iforest_classifier = IsolationForest()

iforest_parameters ={'contamination' : [0.01, 0.04, 0.08, 0.12, 0.16] , 'n_estimators': [4, 8, 12,24,32, 64]}
#  'learning_rate' : [0.01,0.02,0.03],
#  'gamma': [0.5, 1, 2], 'random_state':[99]}
elapsed_time_iforest = []

iforest_parameters = {'n_estimators': list(range(100, 200, 50)), 
              'max_samples': list(range(100, 110, 5)), 
              'contamination': [0.1, 0.2], 
              'max_features': [5,15], 
              'bootstrap': [True, False], 
              }


start_time = datetime.now()
iforest_grid_search = GridSearchCV(iforest_classifier, iforest_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_iforest_classifier = iforest_grid_search.fit(x_train_only)

# iforest_grid_search = GridSearchCV(iforest_classifier, xgb_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
# iforest_classifier.fit(x_train_only)
elapsed_time_iforest.append((datetime.now()-start_time).total_seconds())

Fitting 7 folds for each of 32 candidates, totalling 224 fits
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, contamination=0.1, max_features=5, max_samples=100, n_estimators=100; total time=   0.0s


...

[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=100; total time=   0.0s
[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=150; total time=   0.1s
[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=150; total time=   0.1s


c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=150; total time=   0.1s
[CV] END bootstrap=False, contamination=0.1, max_features=15, max_samples=105, n_estimators=150; total time=   0.1s


In [ ]:
# y_pred = xgb_classifier.predict(x_test)
start_time = datetime.now()
y_pred_c = best_iforest_classifier.predict(x_test)
elapsed_time_iforest.append((datetime.now()-start_time).total_seconds())


acc_iforest = accuracy_score(y_test_a, y_pred_c)

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred_c))
print("best parameters ", iforest_grid_search.best_params_)
print('Accuracy ', acc_iforest)
print('elapsed time ', elapsed_time_iforest)

Confusion Matrix
 [[49 18]
 [ 5 42]]
best parameters  {'bootstrap': True, 'contamination': 0.1, 'max_features': 5, 'max_samples': 100, 'n_estimators': 100}
Accuracy  0.7982456140350878
elapsed time  [25.361736, 0.02447]


# Autoencoder for Anomaly Detection

In [ ]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)



class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

train_data = TrainData(torch.FloatTensor(x_train_only), 
                       torch.FloatTensor(x_train_only))

test_data = TestData(torch.FloatTensor(x_test))
valid_data = TrainData(torch.FloatTensor(x_valid), torch.FloatTensor(x_valid))

train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)
valid_loader = DataLoader(dataset=valid_data, batch_size=2)

In [ ]:
EPOCHS = 600 
BATCH_SIZE = 128 
LEARNING_RATE = 0.001

NUM_INPUT = x_train.shape[1]
NUM_OUTPUT = x_train.shape[1] 
NUM_1ST_HIDDEN = 32 
NUM_2ND_HIDDEN = 8 
NUM_1ST_DROPOUT = 0.1
NUM_2ND_DROPOUT = 0.1
NUM_3RD_DROPOUT = 0.1

In [ ]:
class BasicClassification(nn.Module):
    def __init__(self) -> None:
        super(BasicClassification, self).__init__()

        self.layer_1 = nn.Linear(NUM_INPUT, NUM_1ST_HIDDEN)
        self.layer_2 = nn.Linear(NUM_1ST_HIDDEN, NUM_2ND_HIDDEN)
        self.layer_3 = nn.Linear(NUM_2ND_HIDDEN, NUM_1ST_HIDDEN)
        self.layer_4 = nn.Linear(NUM_1ST_HIDDEN, NUM_INPUT)

        # self.actvation = nn.ReLU()
        self.actvation_1 = nn.SELU()
        self.actvation_2 = nn.SELU()
        self.actvation_3 = nn.SELU()

        # self.actvation_1 = nn.ReLU()
        # self.actvation_2 = nn.ReLU()
        # self.actvation_3 = nn.ReLU()

        self.dropout_1 = nn.Dropout(p=NUM_1ST_DROPOUT)
        self.dropout_2 = nn.Dropout(p=NUM_2ND_DROPOUT)
        self.dropout_3 = nn.Dropout(p=NUM_3RD_DROPOUT)
        self.batchnorm_1 = nn.BatchNorm1d(NUM_1ST_HIDDEN)
        self.batchnorm_2 = nn.BatchNorm1d(NUM_2ND_HIDDEN)
        self.batchnorm_3 = nn.BatchNorm1d(NUM_1ST_HIDDEN)
    
    def forward(self, inputs):
        x = self.actvation_1(self.layer_1(inputs))
        # x = self.batchnorm_1(x)
        # x = self.dropout_1(x)
        x = self.actvation_2(self.layer_2(x))
        # x = self.batchnorm_2(x)
        # x = self.dropout_2(x)

        x = self.actvation_3(self.layer_3(x))
        # x = self.batchnorm_3(x)
        # x = self.dropout_3(x)

        # x = self.actvation_4(self.layer_4(x))
        # x = self.batchnorm_3(x)
        # x = self.dropout_3(x)


        x = self.layer_4(x)

        return x


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
model = BasicClassification()
model.to(device)

print(model)


# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BasicClassification(
  (layer_1): Linear(in_features=30, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=8, bias=True)
  (layer_3): Linear(in_features=8, out_features=32, bias=True)
  (layer_4): Linear(in_features=32, out_features=30, bias=True)
  (actvation_1): SELU()
  (actvation_2): SELU()
  (actvation_3): SELU()
  (dropout_1): Dropout(p=0.1, inplace=False)
  (dropout_2): Dropout(p=0.1, inplace=False)
  (dropout_3): Dropout(p=0.1, inplace=False)
  (batchnorm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [ ]:
best_model  = None
best_loss_on_valid = 999999999
num_train_data = len(train_loader)
num_eval_data = len(valid_loader)


elapsed_time_basic_ann = []

start_time = datetime.now()


for epoch in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0

    eval_epoch_loss = 0
    eval_epoch_acc = 0

    model.train()
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)
        # acc = calc_accuracy(y_pred, y_batch)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        # epoch_acc += acc.item()
    
    if epoch % 10 == 0:
        
        with torch.no_grad():
            model.eval()
            for x, y in valid_loader:
                x = x.to(device)
                y = y.to(device)

                output = model(x)

                eval_loss = criterion(output, y)
                # eval_acc = calc_accuracy(output, y)

                eval_epoch_loss += eval_loss.item()
                # eval_epoch_acc += eval_acc.item()
        
        if best_loss_on_valid >= (eval_epoch_loss/num_eval_data):
            best_loss_on_valid = (eval_epoch_loss/num_eval_data)
            best_model = copy.deepcopy(model)
            print("Best Model is copied - Best Loss : ", best_loss_on_valid)
        


        print(f"Epoch {epoch+0:03}: : Loss: T_{epoch_loss/num_train_data:.3f} V_{eval_epoch_loss/num_eval_data:.3f} | Acc: T_{epoch_acc/num_train_data:.3f}) V_{eval_epoch_acc/num_eval_data:.3f}")


elapsed_time_basic_ann.append((datetime.now()-start_time).total_seconds())

Best Model is copied - Best Loss :  0.02352614022548432
Epoch 010: : Loss: T_0.020 V_0.024 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.018237976701525244
Epoch 020: : Loss: T_0.016 V_0.018 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.013874301676759902
Epoch 030: : Loss: T_0.013 V_0.014 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.010672355556617613
Epoch 040: : Loss: T_0.009 V_0.011 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.009111138080935116
Epoch 050: : Loss: T_0.007 V_0.009 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.008002929900691885
Epoch 060: : Loss: T_0.006 V_0.008 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.006882169749587774
Epoch 070: : Loss: T_0.005 V_0.007 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.00611643719173077
Epoch 080: : Loss: T_0.005 V_0.006 | Acc: T_0.000) V_0.000
Best Model is copied - Best Loss :  0.005557275531590794
Epoch 090: : Loss

In [ ]:

# with torch.no_grad():
# model.eval()
best_model.eval()
data = torch.from_numpy(x_test).float().to(device)
# answer = torch.from_numpy(y_test_onehot).float().to(device)


# data = torch.from_numpy(x_train).float().to(device)
# answer = torch.from_numpy(y_train_onehot).float().to(device)


# data = torch.from_numpy(x_valid).float().to(device)
# answer = torch.from_numpy(y_valid_onehot).float().to(device)

start_time = datetime.now()
output = best_model(data)
# acc_basic_ann = calc_accuracy(output, answer)
elapsed_time_basic_ann.append((datetime.now()-start_time).total_seconds())

# print('Accuracy ', acc_basic_ann)
print('elapsed time ', elapsed_time_basic_ann)

elapsed time  [11.57845, 0.0]


In [ ]:

output_num = output.detach().cpu().numpy()



In [ ]:
output_num[0]

array([ 1.4471334e-01,  2.4935968e-01,  1.3014387e-01,  2.2495998e-02,
        3.1087187e-01,  2.9183418e-02, -5.2451141e-02,  7.1492456e-03,
        3.0852810e-01,  1.9560257e-01,  1.3573512e-01,  1.6761552e-01,
        1.2036891e-01,  4.5298927e-02,  9.0290651e-02, -1.9698143e-02,
        1.9924324e-02,  1.5515509e-01,  1.6489801e-01,  1.6763240e-02,
        1.2468188e-01,  2.4531257e-01,  1.0256621e-01, -1.2417793e-02,
        2.2487807e-01,  2.4682283e-04,  1.5664451e-02,  1.5848196e-01,
        2.9761344e-01,  8.5421361e-02], dtype=float32)

In [ ]:
x_test[0]

array([0.17275894, 0.17145756, 0.16688007, 0.09094824, 0.27471337,
       0.07729587, 0.0022814 , 0.0146173 , 0.32354511, 0.24079006,
       0.03726236, 0.19823555, 0.03637563, 0.01544272, 0.13611978,
       0.03659086, 0.00245884, 0.05571131, 0.12829966, 0.03921894,
       0.14019796, 0.26759062, 0.12838221, 0.0624408 , 0.25774699,
       0.06214833, 0.00471624, 0.05728522, 0.29698266, 0.12554113])

In [ ]:
result_reconstruct = abs(x_test - output_num).sum(axis=1)
print(len(result_reconstruct))
print(len(x_test))
print(result_reconstruct)

114
114
[1.3249706  0.70405705 0.83391559 2.98150279 1.23020378 0.80065515
 0.45200274 2.00750458 0.96713778 1.50392395 0.95116067 1.25978626
 0.43956438 0.68474542 0.74832416 0.7977418  1.14017677 3.29537973
 1.04457215 1.14596386 0.51280209 3.16478602 1.08822125 1.27517696
 0.51885977 0.97479153 0.67361697 1.08648562 0.81417664 0.6267367
 0.88058375 1.06832489 0.61167945 0.67063647 0.70927887 1.08923339
 0.93611968 1.50294474 0.96544419 1.3299273  0.43366944 0.81223628
 0.95354123 0.61384636 0.84748778 0.80807147 0.72287669 0.78800818
 0.5886418  0.5840312  1.21028083 0.87167369 1.24216815 0.85202711
 0.89894232 0.82502871 0.44397422 0.88866477 0.99356976 0.51796896
 1.29391577 1.19714214 0.6840033  1.21925381 2.52924049 0.67751217
 0.90245822 1.90088875 1.33024771 0.7324526  0.69500224 1.63572239
 0.65181822 1.00356511 1.11282566 1.09870246 1.51986326 0.82550863
 3.42700237 0.88309476 2.1209164  3.20471407 0.70638289 0.85635881
 0.61084135 0.64730358 0.75711459 1.04143053 2.05456656

In [ ]:
result_class = result_reconstruct.copy()
result_class[result_reconstruct > THRESHOLD_FOR_AUTOENCODER] = -1
result_class[result_reconstruct <= THRESHOLD_FOR_AUTOENCODER] = 1

In [ ]:
# result_class
acc_ae = accuracy_score(y_test_a, result_class)

print('Confusion Matrix\n', confusion_matrix(y_test_a, result_class))
print('Accuracy ', acc_ae)

Confusion Matrix
 [[45 22]
 [19 28]]
Accuracy  0.6403508771929824


In [ ]:
len(output_num)

114

# Mixture of Gaussian

In [ ]:
gmm_classifier = GaussianMixture()

# lgbm_classifier=lgbm_classifier.fit(x_train, y_train)
# y_pred = lgbm_classifier.predict(x_test)


gmm_parameters ={'n_components' : [1, 2, 3,4,5,6, 7] , 'max_iter': [int(1e2), int(1e3), int(1e6)]}
#  'learning_rate' : [0.01,0.02,0.03],
#  'gamma': [0.5, 1, 2], 'random_state':[99]}

elapsed_time_gmm= []

start_time = datetime.now()
gmm_grid_search = GridSearchCV(gmm_classifier, gmm_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_gmm_classifier = gmm_grid_search.fit(x_train_only)
elapsed_time_gmm.append((datetime.now()-start_time).total_seconds())




# acc_lgbm = accuracy_score(y_test, y_pred)

# print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
# print("best parameters ", lgbm_grid_search.best_params_)
# print('Accuracy ', acc_lgbm)
# print('elapsed time ', elapsed_time_lgbm)


Fitting 7 folds for each of 21 candidates, totalling 147 fits
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=1; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n_components=2; total time=   0.0s
[CV] END .......................max_iter=100, n

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=3; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END .......................max_iter=100, n_components=4; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END .......................max_iter=100, n_components=5; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=6; total time=   0.0s
[CV] END .......................max_iter=100, n_components=7; total time=   0.0s
[CV] END .......................max_iter=100, n_components=7; total time=   0.0s
[CV] END .......................max_iter=100, n_components=7; total time=   0.0s
[CV] END .......................max_iter=100, n_components=7; total time=   0.0s
[CV] END ...................

...

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=1; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=2; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=2; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=2; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=2; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=2; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=5; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................max_iter=1000000, n_components=6; total time=   0.0s
[CV] END ...................

c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\venv_python_3.10\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\venv_python_3.10\lib\site-packa

In [ ]:
start_time = datetime.now()
y_pred_c = best_gmm_classifier.predict(x_test)
elapsed_time_gmm.append((datetime.now()-start_time).total_seconds())

In [ ]:
y_pred_c

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64)

In [ ]:
print(len(np.argwhere(y_test_a == -1)))

67


In [ ]:
densities = best_gmm_classifier.score_samples(x_test)
density_threshold = np.percentile(densities, THRESHOLD_FOR_DENSITY)
anomalies = np.argwhere(densities < density_threshold)
print(len(anomalies))
# print(anomalies)

# print(density_threshold)
# print(len(anomalies))
# print(np.argwhere(y_test_c == -1))
real_anomaly = np.argwhere(y_test_a == -1)
# print(real_)

y_pred_anomalies = y_test_a.copy()
y_pred_anomalies[densities < density_threshold] = -1
y_pred_anomalies[densities >= density_threshold] = 1


acc_iforest = accuracy_score(y_test_a, y_pred_anomalies)

print('Confusion Matrix\n', confusion_matrix(y_test_a, y_pred_anomalies))
print("best parameters ", best_gmm_classifier.best_params_)
print('Accuracy ', acc_iforest)
print('elapsed time ', elapsed_time_gmm)

# accuracy_score(y_test_c, y_pred_anomalies)

80
Confusion Matrix
 [[56 11]
 [24 23]]
best parameters  {'max_iter': 100, 'n_components': 1}
Accuracy  0.6929824561403509
elapsed time  [1.440888, 0.0]


In [ ]:
# y_pred_anomalies

In [ ]:
y_test_a[anomalies]
y_test_a[real_anomaly]

array([[-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1]])

In [146]:
anomalies.shape

(4, 10)